In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.17.2
pandas 0.25.1
sklearn 0.21.3
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [3]:
train_file = "./train.csv"
eval_file = "./eval.csv"

train_df = pd.read_csv(train_file)
eval_df = pd.read_csv(eval_file)

display(train_df)

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
...,...,...,...,...,...,...,...,...,...,...
622,0,male,28.0,0,0,10.5000,Second,unknown,Southampton,y
623,0,male,25.0,0,0,7.0500,Third,unknown,Southampton,y
624,1,female,19.0,0,0,30.0000,First,B,Southampton,y
625,0,female,28.0,1,2,23.4500,Third,unknown,Southampton,n


In [4]:
eval_df

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,35.0,0,0,8.0500,Third,unknown,Southampton,y
1,0,male,54.0,0,0,51.8625,First,E,Southampton,y
2,1,female,58.0,0,0,26.5500,First,C,Southampton,y
3,1,female,55.0,0,0,16.0000,Second,unknown,Southampton,y
4,1,male,34.0,0,0,13.0000,Second,D,Southampton,y
...,...,...,...,...,...,...,...,...,...,...
259,1,female,25.0,0,1,26.0000,Second,unknown,Southampton,n
260,0,male,33.0,0,0,7.8958,Third,unknown,Southampton,y
261,0,female,39.0,0,5,29.1250,Third,unknown,Queenstown,n
262,0,male,27.0,0,0,13.0000,Second,unknown,Southampton,y


In [5]:
y_train = train_df.pop('survived')
y_eval = eval_df.pop('survived')


In [6]:
categorical_columns = ['sex', 'n_siblings_spouses', 'parch', 'class',
                       'deck', 'embark_town', 'alone']
numeric_columns = ['age', 'fare']

In [7]:
feature_columns = []

for categorical_column in categorical_columns:
    vocab = train_df[categorical_column].unique()
    print(categorical_column,vocab)
    feature_columns.append(
        tf.feature_column.indicator_column(
            tf.feature_column.categorical_column_with_vocabulary_list(categorical_column,vocab)
        )
    )
    
for c in numeric_columns:
    feature_columns.append(
        tf.feature_column.numeric_column(
            c,dtype=tf.float32)
    )

sex ['male' 'female']
n_siblings_spouses [1 0 3 4 2 5 8]
parch [0 1 2 5 3 4]
class ['Third' 'First' 'Second']
deck ['unknown' 'C' 'G' 'A' 'B' 'D' 'F' 'E']
embark_town ['Southampton' 'Cherbourg' 'Queenstown' 'unknown']
alone ['n' 'y']


In [8]:
def make_dataset(data_df,label_df,epochs=10,shuffle=True,batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices(
        (dict(data_df), label_df)
    )
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [9]:
train_dataset = make_dataset(train_df,y_train,batch_size=5)
for x,y in train_dataset.take(1):
    print(x,y)

{'sex': <tf.Tensor: id=38, shape=(5,), dtype=string, numpy=array([b'female', b'male', b'male', b'male', b'male'], dtype=object)>, 'age': <tf.Tensor: id=30, shape=(5,), dtype=float64, numpy=array([29., 65., 47., 28., 59.])>, 'n_siblings_spouses': <tf.Tensor: id=36, shape=(5,), dtype=int32, numpy=array([1, 0, 0, 0, 0], dtype=int32)>, 'parch': <tf.Tensor: id=37, shape=(5,), dtype=int32, numpy=array([0, 1, 0, 0, 0], dtype=int32)>, 'fare': <tf.Tensor: id=35, shape=(5,), dtype=float64, numpy=array([26.    , 61.9792,  7.25  , 56.4958, 13.5   ])>, 'class': <tf.Tensor: id=32, shape=(5,), dtype=string, numpy=array([b'Second', b'First', b'Third', b'Third', b'Second'], dtype=object)>, 'deck': <tf.Tensor: id=33, shape=(5,), dtype=string, numpy=array([b'unknown', b'B', b'unknown', b'unknown', b'unknown'], dtype=object)>, 'embark_town': <tf.Tensor: id=34, shape=(5,), dtype=string, numpy=
array([b'Southampton', b'Cherbourg', b'Southampton', b'Southampton',
       b'Southampton'], dtype=object)>, 'alon

In [10]:
# keras.layers.DenseFeature 
for x,y in train_dataset.take(1):
    age_column = feature_columns[7]
    gender_column = feature_columns[0]
    print(keras.layers.DenseFeatures(age_column)(x).numpy())
    print(keras.layers.DenseFeatures(gender_column)(x).numpy())


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[28.]
 [40.]
 [35.]
 [24.]
 [31.]]

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base 

In [11]:
# keras.layers.DenseFeature
for x, y in train_dataset.take(1):
    print(keras.layers.DenseFeatures(feature_columns)(x).numpy())


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[34.      0.      1.      1.      0.      0.      1.      0.      0.
   0.      0.      0.      0.      0.      1.      0.      0.      0.
   6.4958  0.      1.      0.      0.      0.      0.      0.      1.
   0.      0.      0.      0.      0.      1.      0.    ]
 [35.      1.      0.      0.      1.      0.      0.      1.      0.
   0.      0.      0.      0.      0.      1.      0.      0.      0.
  83.475   1.      0.      0.      0.      0.      0.      0.      1.
   0.      0.      0.      0.

In [15]:
model = keras.Sequential([
    keras.layers.DenseFeatures(feature_columns)
    ,keras.layers.Dense(100,activation='relu')
    ,keras.layers.Dense(100,activation='relu')
    ,keras.layers.Dense(2,activation='softmax')
])
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer = keras.optimizers.SGD(lr=0.01),
              metrics = ['accuracy'])

In [17]:
# 1. model fit
# 2. model -> estimator -> train

dataset = make_dataset(train_df,y_train,epochs=10)
eval_dataset = make_dataset(eval_df, y_eval, epochs = 1, shuffle = False)

history = model.fit(train_dataset,
                    validation_data = eval_dataset,
                    steps_per_epoch = 19,
                    validation_steps = 8,
                    epochs = 10)


Train for 19 steps, validate for 8 steps
Epoch 1/10
19/19 [==============================] - 0s 10ms/step - loss: 0.6773 - accuracy: 0.6211 - val_loss: 0.6165 - val_accuracy: 0.6562
Epoch 2/10
19/19 [==============================] - 0s 11ms/step - loss: 0.6381 - accuracy: 0.5474 - val_loss: 0.6137 - val_accuracy: 0.6406
Epoch 3/10
19/19 [==============================] - 0s 10ms/step - loss: 0.5844 - accuracy: 0.6947 - val_loss: 0.6060 - val_accuracy: 0.6680
Epoch 4/10
19/19 [==============================] - 0s 12ms/step - loss: 0.5779 - accuracy: 0.7789 - val_loss: 0.5977 - val_accuracy: 0.6875
Epoch 5/10
19/19 [==============================] - 0s 10ms/step - loss: 0.6013 - accuracy: 0.7053 - val_loss: 0.6300 - val_accuracy: 0.6367
Epoch 6/10
19/19 [==============================] - 0s 12ms/step - loss: 0.5507 - accuracy: 0.7368 - val_loss: 0.6284 - val_accuracy: 0.6992
Epoch 7/10
19/19 [==============================] - 0s 11ms/step - loss: 0.6512 - accuracy: 0.6000 - val_loss: 0.

In [18]:
estimator = keras.estimator.model_to_estimator(model)
estimator.train(input_fn = lambda : make_dataset(train_df,y_train,epochs=100))

AttributeError: 'Sequential' object has no attribute 'estimator'